----
# <b>ALTEGRAD 2023
# Lab 6: Deep Learning for Graphs (2/2)
# MARENGO Matteo | matteo.marengo@ens-paris-saclay.fr
# <b>PART 1 - Graph Neural Networks for Graph-Level Tasks
----

<i>Graph level means that each sample is a graph and not a node</i>

## <b>1. Dataset Generation</b>
The dataset will contain two types of graphs:
- instances of the $G(n,0.2)$ Erdos-Rényi random model
- instances of the $G(n,0.4)$ Erdos-Rényi random model
- $n \in [10,20]$

In the $G(n,p)$ model, a graph is constructed by connecting nodes randomly.
### <b>TASK 1</b>
Generate 50 graphs using the $G(n,0.2)$ model and 50 graphs using the $G(n,0.4)$ model.<br>
<i>Hint: Use the fast_gnp_random_graph() function of NetworkX</i>



In [1]:
"""
Deep Learning on Graphs - ALTEGRAD - Nov 2023
utils.py - 1/3
Matteo MARENGO - matteo.marengo@ens-paris-saclay.fr
"""

import networkx as nx
import numpy as np
import torch
from random import randint

def create_dataset():
    Gs = list()
    y = list()

    ############## Task 1
    # Generate the dataset
    # 50 graphs using the G(n,p1) model with p1=0.2
    # 50 graphs using the G(n,p2) model with p2=0.4
    
    ##################
    p1 = 0.2
    p2 = 0.4

    for i in range(50):
        n = randint(10,20)
        Gs.append(nx.fast_gnp_random_graph(n,p1))
        y.append(0)

        n = randint(10,20)
        Gs.append(nx.fast_gnp_random_graph(n,p2))
        y.append(1)
    ##################

    return Gs, y


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse_coo_tensor(indices, values, shape)

## <b> 2. Implementation of Graph Neural Network </b>
### <b>TASK 2</b>
GNN model that consists of:
- two message passing layers
- sum readout function
- two fully-connected layers


In [2]:
"""
Deep Learning on Graphs - ALTEGRAD - Nov 2023
models.py - 2/3
Matteo MARENGO - matteo.marengo@ens-paris-saclay.fr
"""

import torch
import torch.nn as nn
import torch.nn.functional as F

class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim_1, hidden_dim_2, hidden_dim_3, n_class):
        super(GNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim_1)
        self.fc2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.fc3 = nn.Linear(hidden_dim_2, hidden_dim_3)
        self.fc4 = nn.Linear(hidden_dim_3, n_class)
        self.relu = nn.ReLU()

    def forward(self, x_in, adj, idx):
        
        ############## Task 2
    
        ##################
        # a message passing layer with h1 hidden units followed by a ReLU activation function
        x = self.fc1(x_in)
        x = self.relu(torch.mm(adj, x))

        # a message passing layer with h2 hidden units followed by a ReLU activation function
        x = self.fc2(x)
        x = self.relu(torch.mm(adj, x))

        ##################
        # a readout function that computes the sum of the node embeddings for each graph in the batch
        idx = idx.unsqueeze(1).repeat(1, x.size(1))
        out = torch.zeros(torch.max(idx)+1, x.size(1), device=x_in.device)
        out = out.scatter_add_(0, idx, x) 
        
        ##################
        # your code here #
        # a fully connected layer with h3 hidden units 
        out = self.relu(self.fc3(out))
        # a fully connected layer with nclass hidden units 
        out = self.fc4(out)

        ##################

        return F.log_softmax(out, dim=1)


## <b> 3. Graph Classification
### <b>TASK 3

In [3]:
"""
Deep Learning on Graphs - ALTEGRAD - Nov 2023
gnn.py - 3/3
Matteo MARENGO - matteo.marengo@ens-paris-saclay.fr
"""

import time
import networkx as nx
import numpy as np
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch import optim

# from models import GNN
# Sfrom utils import create_dataset, sparse_mx_to_torch_sparse_tensor

# Initializes device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Hyperparameters
epochs = 200
batch_size = 8
n_hidden_1 = 16
n_hidden_2 = 32
n_hidden_3 = 32
learning_rate = 0.01

# Generates synthetic dataset
Gs, y = create_dataset()
n_class = np.unique(y).size
print('n class: {}'.format(n_class))

# Splits the dataset into a training and a test set
G_train, G_test, y_train, y_test = train_test_split(Gs, y, test_size=0.1)

N_train = len(G_train)
N_test = len(G_test)

# Initializes model and optimizer
model = GNN(1, n_hidden_1, n_hidden_2, n_hidden_3, n_class).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

# Trains the model
for epoch in range(epochs):
    t = time.time()
    model.train()
    
    train_loss = 0
    correct = 0
    count = 0
    for i in range(0, N_train, batch_size):
        adj_batch = list()
        idx_batch = list()
        y_batch = list()

        ############## Task 3
        
        ##################
        # your code here #

        for j in range(i, min(N_train, i+batch_size)):
            n = G_train[j].number_of_nodes()
            adj_batch.append(nx.adjacency_matrix(G_train[j]) + sp.identity(n))
            idx_batch.extend([j-i]*n)
            y_batch.append(y_train[j])
        # create a sparse block diagonal matrix from the adjacency matrices of the graphs in the batch
        adj_batch = sp.block_diag(adj_batch)
        # a matrix of size (N,1) where N is the number of nodes in the batch
        # it countains thefeatures of the nodes in the batch
        # here we use constant features equal to 1
        features_batch = np.ones((adj_batch.shape[0],1))

        # convert the numpy arrays to a torch sparse tensor
        adj_batch = sparse_mx_to_torch_sparse_tensor(adj_batch).to(device)
        features_batch = torch.FloatTensor(features_batch).to(device)
        idx_batch = torch.LongTensor(idx_batch).to(device)
        y_batch = torch.LongTensor(y_batch).to(device)
        ##################
        
        optimizer.zero_grad()
        output = model(features_batch, adj_batch, idx_batch)
        loss = loss_function(output, y_batch)
        train_loss += loss.item() * output.size(0)
        count += output.size(0)
        preds = output.max(1)[1].type_as(y_batch)
        correct += torch.sum(preds.eq(y_batch).double())
        loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(train_loss / count),
              'acc_train: {:.4f}'.format(correct / count),
              'time: {:.4f}s'.format(time.time() - t))
        
print('Optimization finished!')

# Evaluates the model
model.eval()
test_loss = 0
correct = 0
count = 0
for i in range(0, N_test, batch_size):
    adj_batch = list()
    idx_batch = list()
    y_batch = list()

    ############## Task 3
    
    ##################
    # your code here #

    for j in range(i, min(N_train, i+batch_size)):
        # extract the adjacency matrix of the j-th graph in the batch
        n = G_train[j].number_of_nodes()
        adj_batch.append(nx.adjacency_matrix(G_train[j]) + sp.identity(n))
        # a vector that contains the indices of the graphs in the batch
        idx_batch.extend([j-i]*n)
        # a vector that contains the class labels of the nodes in the batch
        y_batch.append(y_train[j])

    # create a sparse block diagonal matrix from the adjacency matrices of the graphs in the batch
    adj_batch = sp.block_diag(adj_batch)
    features_batch = np.ones((adj_batch.shape[0],1))

    adj_batch = sparse_mx_to_torch_sparse_tensor(adj_batch).to(device)
    features_batch = torch.FloatTensor(features_batch).to(device)
    idx_batch = torch.LongTensor(idx_batch).to(device)
    y_batch = torch.LongTensor(y_batch).to(device)
    ##################

    output = model(features_batch, adj_batch, idx_batch)
    loss = loss_function(output, y_batch)
    test_loss += loss.item() * output.size(0)
    count += output.size(0)
    preds = output.max(1)[1].type_as(y_batch)
    correct += torch.sum(preds.eq(y_batch).double())

print('loss_test: {:.4f}'.format(test_loss / count),
      'acc_test: {:.4f}'.format(correct / count),
      'time: {:.4f}s'.format(time.time() - t))


c:\Users\matte\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\cuda\__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


n class: 2


c:\Users\matte\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch: 0001 loss_train: 2.7542 acc_train: 0.6000 time: 0.1476s
Epoch: 0011 loss_train: 0.5967 acc_train: 0.7444 time: 0.0782s
Epoch: 0021 loss_train: 0.3640 acc_train: 0.7889 time: 0.0723s
Epoch: 0031 loss_train: 0.3258 acc_train: 0.8111 time: 0.0733s
Epoch: 0041 loss_train: 0.3564 acc_train: 0.8000 time: 0.0728s
Epoch: 0051 loss_train: 0.3434 acc_train: 0.7889 time: 0.0852s
Epoch: 0061 loss_train: 0.3208 acc_train: 0.8444 time: 0.0673s
Epoch: 0071 loss_train: 0.2892 acc_train: 0.8556 time: 0.0728s
Epoch: 0081 loss_train: 0.3230 acc_train: 0.8111 time: 0.0728s
Epoch: 0091 loss_train: 0.2747 acc_train: 0.8667 time: 0.0764s
Epoch: 0101 loss_train: 0.2619 acc_train: 0.8778 time: 0.0811s
Epoch: 0111 loss_train: 0.2577 acc_train: 0.8889 time: 0.0734s
Epoch: 0121 loss_train: 0.2204 acc_train: 0.9000 time: 0.0812s
Epoch: 0131 loss_train: 0.2185 acc_train: 0.9000 time: 0.0831s
Epoch: 0141 loss_train: 0.2072 acc_train: 0.9333 time: 0.0890s
Epoch: 0151 loss_train: 0.2153 acc_train: 0.9222 time: 